# 📊 Ethereum Fraud Detection - Complete Data Analysis & Visualization

**Author:** Nopal  
**Date:** November 5, 2025  
**Thesis:** Ethereum Fraud Detection using XGBoost + SHAP + Network Graph Analysis

---

## 📋 Notebook Overview

This notebook provides comprehensive visualization of the entire ML pipeline:

1. **Raw Data Exploration** - Transaction data from Ethereum Mainnet
2. **Feature Engineering Analysis** - Temporal, Value, Gas, Account Behavior
3. **Network Graph Visualization** - Centrality, Community Detection, Risk Propagation
4. **Model Training Analysis** - XGBoost hyperparameters, training process
5. **SHAP Explainability** - Global & local feature importance
6. **Evaluation Metrics** - Confusion Matrix, PR-AUC, ROC curves

---

## 🔧 Setup & Imports

In [ ]:
# Standard libraries
import sys
import warnings
from pathlib import Path
from datetime import datetime

# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Network analysis
import networkx as nx

# ML libraries
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import xgboost as xgb
import shap

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

# Import project modules
from config import RAW_DATA_DIR, PROCESSED_DATA_DIR, MODEL_DIR

# Suppress warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.6f}'.format)

print("✅ All libraries imported successfully!")
print(f"📂 Working directory: {Path.cwd()}")
print(f"📊 Notebook executed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

---
## 📦 Part 1: Raw Data Exploration

Load and explore the raw transaction data fetched from Ethereum Mainnet.

In [ ]:
# Load raw transaction data
raw_file = RAW_DATA_DIR / "transactions_raw.csv"
df_raw = pd.read_csv(raw_file)

# Convert timestamp
df_raw['timestamp'] = pd.to_datetime(df_raw['timestamp'])

print(f"📊 Dataset Shape: {df_raw.shape}")
print(f"📅 Date Range: {df_raw['timestamp'].min()} to {df_raw['timestamp'].max()}")
print(f"\n🔍 First 5 rows:")
df_raw.head()

In [ ]:
# Dataset overview
print("=" * 60)
print("📋 DATASET INFORMATION")
print("=" * 60)
print(f"Total Transactions: {len(df_raw):,}")
print(f"Unique Senders (from): {df_raw['from'].nunique():,}")
print(f"Unique Receivers (to): {df_raw['to'].nunique():,}")
print(f"Unique Blocks: {df_raw['blockNumber'].nunique():,}")
print(f"\nFraud Distribution:")
print(df_raw['is_fraud'].value_counts())
print(f"\nFraud Rate: {df_raw['is_fraud'].mean() * 100:.2f}%")
print("\n" + "=" * 60)

### 📈 Visualization 1.1: Transaction Timeline

In [ ]:
# Transaction timeline
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Daily Transaction Count', 'Cumulative Transactions'),
    vertical_spacing=0.15
)

# Daily transaction count
daily_txs = df_raw.groupby(df_raw['timestamp'].dt.date).size()
fig.add_trace(
    go.Scatter(x=daily_txs.index, y=daily_txs.values, mode='lines+markers',
               name='Daily Transactions', line=dict(color='#3498db', width=2)),
    row=1, col=1
)

# Cumulative transactions
cumulative_txs = daily_txs.cumsum()
fig.add_trace(
    go.Scatter(x=cumulative_txs.index, y=cumulative_txs.values, 
               fill='tonexty', name='Cumulative', 
               line=dict(color='#2ecc71', width=2)),
    row=2, col=1
)

fig.update_layout(height=700, title_text="Transaction Timeline Analysis", showlegend=True)
fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_yaxes(title_text="Cumulative Count", row=2, col=1)
fig.show()

### 📈 Visualization 1.2: Transaction Value Distribution

In [ ]:
# Value distribution analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Value distribution (log scale)
non_zero_values = df_raw[df_raw['value_eth'] > 0]['value_eth']
axes[0, 0].hist(np.log10(non_zero_values), bins=50, color='#3498db', alpha=0.7, edgecolor='black')
axes[0, 0].set_xlabel('log10(Value in ETH)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Transaction Value Distribution (Log Scale)', fontsize=14, fontweight='bold')
axes[0, 0].grid(alpha=0.3)

# 2. Box plot by hour
df_raw['hour'] = df_raw['timestamp'].dt.hour
hourly_values = df_raw[df_raw['value_eth'] > 0].groupby('hour')['value_eth'].median()
axes[0, 1].bar(hourly_values.index, hourly_values.values, color='#2ecc71', alpha=0.7, edgecolor='black')
axes[0, 1].set_xlabel('Hour of Day')
axes[0, 1].set_ylabel('Median Value (ETH)')
axes[0, 1].set_title('Median Transaction Value by Hour', fontsize=14, fontweight='bold')
axes[0, 1].grid(alpha=0.3)

# 3. Gas price distribution
axes[1, 0].hist(df_raw['gasPrice'] / 1e9, bins=50, color='#e74c3c', alpha=0.7, edgecolor='black')
axes[1, 0].set_xlabel('Gas Price (Gwei)')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Gas Price Distribution', fontsize=14, fontweight='bold')
axes[1, 0].grid(alpha=0.3)

# 4. Gas used distribution
axes[1, 1].hist(df_raw['gasUsed'], bins=50, color='#f39c12', alpha=0.7, edgecolor='black')
axes[1, 1].set_xlabel('Gas Used')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Gas Used Distribution', fontsize=14, fontweight='bold')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../models/viz_01_value_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Saved: viz_01_value_distribution.png")

### 📈 Visualization 1.3: Temporal Patterns

In [ ]:
# Temporal patterns
df_raw['day_of_week'] = df_raw['timestamp'].dt.day_name()
df_raw['hour'] = df_raw['timestamp'].dt.hour

# Create heatmap data
heatmap_data = df_raw.groupby(['day_of_week', 'hour']).size().unstack(fill_value=0)
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
heatmap_data = heatmap_data.reindex(day_order)

# Plot heatmap
plt.figure(figsize=(16, 6))
sns.heatmap(heatmap_data, cmap='YlOrRd', annot=True, fmt='g', 
            cbar_kws={'label': 'Transaction Count'})
plt.title('Transaction Activity Heatmap (Day of Week vs Hour)', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Hour of Day', fontsize=12)
plt.ylabel('Day of Week', fontsize=12)
plt.tight_layout()
plt.savefig('../models/viz_02_temporal_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Saved: viz_02_temporal_heatmap.png")

---
## 🔨 Part 2: Feature Engineering Analysis

Analyze the engineered features before training.

In [ ]:
# Load processed features
features_file = PROCESSED_DATA_DIR / "features.csv"
df_features = pd.read_csv(features_file)

print(f"📊 Features Shape: {df_features.shape}")
print(f"\n🔍 Feature Columns:")
print(df_features.columns.tolist())
print(f"\n📈 Features Summary:")
df_features.describe()

### 📈 Visualization 2.1: Feature Correlation Matrix

In [ ]:
# Select numeric features only (exclude metadata)
numeric_features = df_features.select_dtypes(include=[np.number]).columns.tolist()
exclude_cols = ['blockNumber', 'timeStamp', 'nonce', 'transactionIndex', 'value', 'gas', 'gasPrice', 'gasUsed', 'isError']
feature_cols = [col for col in numeric_features if col not in exclude_cols]

# Compute correlation matrix
corr_matrix = df_features[feature_cols].corr()

# Plot correlation heatmap
plt.figure(figsize=(20, 16))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', 
            cmap='coolwarm', center=0, square=True, linewidths=1,
            cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Matrix', fontsize=18, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig('../models/viz_03_correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Saved: viz_03_correlation_matrix.png")
print(f"\n🔍 Top 10 Strongest Correlations:")
corr_pairs = corr_matrix.unstack()
corr_pairs = corr_pairs[corr_pairs < 1]
print(corr_pairs.abs().sort_values(ascending=False).head(10))

### 📈 Visualization 2.2: Feature Distributions

In [ ]:
# Plot feature distributions
n_features = len(feature_cols[:16])  # Plot first 16 features
n_cols = 4
n_rows = (n_features + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, n_rows * 4))
axes = axes.flatten()

for idx, feature in enumerate(feature_cols[:16]):
    data = df_features[feature].dropna()
    axes[idx].hist(data, bins=50, color='skyblue', alpha=0.7, edgecolor='black')
    axes[idx].set_title(f'{feature}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Value')
    axes[idx].set_ylabel('Frequency')
    axes[idx].grid(alpha=0.3)
    
    # Add statistics
    mean_val = data.mean()
    median_val = data.median()
    axes[idx].axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_val:.2f}')
    axes[idx].axvline(median_val, color='green', linestyle='--', linewidth=2, label=f'Median: {median_val:.2f}')
    axes[idx].legend(fontsize=8)

# Remove empty subplots
for idx in range(n_features, len(axes)):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.savefig('../models/viz_04_feature_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Saved: viz_04_feature_distributions.png")

### 📈 Visualization 2.3: PCA & t-SNE Dimensionality Reduction

In [ ]:
# Prepare data for dimensionality reduction
X = df_features[feature_cols].fillna(0).replace([np.inf, -np.inf], 0)
y = df_features['is_fraud'].fillna(0)

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# t-SNE (use subset for speed)
sample_size = min(1000, len(X_scaled))
sample_idx = np.random.choice(len(X_scaled), sample_size, replace=False)
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_tsne = tsne.fit_transform(X_scaled[sample_idx])

# Plot
fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# PCA plot
scatter1 = axes[0].scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='RdYlGn_r', 
                           alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
axes[0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)', fontsize=12)
axes[0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)', fontsize=12)
axes[0].set_title('PCA Projection', fontsize=14, fontweight='bold')
axes[0].grid(alpha=0.3)
plt.colorbar(scatter1, ax=axes[0], label='Fraud (1) / Normal (0)')

# t-SNE plot
scatter2 = axes[1].scatter(X_tsne[:, 0], X_tsne[:, 1], c=y.iloc[sample_idx], 
                           cmap='RdYlGn_r', alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
axes[1].set_xlabel('t-SNE Component 1', fontsize=12)
axes[1].set_ylabel('t-SNE Component 2', fontsize=12)
axes[1].set_title(f't-SNE Projection (n={sample_size})', fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3)
plt.colorbar(scatter2, ax=axes[1], label='Fraud (1) / Normal (0)')

plt.tight_layout()
plt.savefig('../models/viz_05_dimensionality_reduction.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Saved: viz_05_dimensionality_reduction.png")
print(f"\nℹ️ PCA Explained Variance: {pca.explained_variance_ratio_.sum()*100:.2f}%")

---
## 🕸️ Part 3: Network Graph Analysis

Visualize the transaction network and graph-based features.

In [ ]:
# Build transaction graph
G = nx.DiGraph()

for _, row in df_raw.iterrows():
    G.add_edge(row['from'], row['to'], weight=row['value_eth'])

print(f"📊 Network Statistics:")
print(f"   Nodes (Addresses): {G.number_of_nodes():,}")
print(f"   Edges (Transactions): {G.number_of_edges():,}")
print(f"   Density: {nx.density(G):.6f}")
print(f"   Connected Components: {nx.number_weakly_connected_components(G)}")

### 📈 Visualization 3.1: Network Graph Centrality

In [ ]:
# Compute centrality measures
degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G, k=100)  # Sample for speed
pagerank = nx.pagerank(G)

# Plot centrality distributions
fig, axes = plt.subplots(1, 3, figsize=(20, 5))

# Degree centrality
degree_values = list(degree_centrality.values())
axes[0].hist(degree_values, bins=50, color='#3498db', alpha=0.7, edgecolor='black')
axes[0].set_xlabel('Degree Centrality', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Degree Centrality Distribution', fontsize=14, fontweight='bold')
axes[0].set_yscale('log')
axes[0].grid(alpha=0.3)

# Betweenness centrality
betweenness_values = list(betweenness_centrality.values())
axes[1].hist(betweenness_values, bins=50, color='#e74c3c', alpha=0.7, edgecolor='black')
axes[1].set_xlabel('Betweenness Centrality', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title('Betweenness Centrality Distribution', fontsize=14, fontweight='bold')
axes[1].set_yscale('log')
axes[1].grid(alpha=0.3)

# PageRank
pagerank_values = list(pagerank.values())
axes[2].hist(pagerank_values, bins=50, color='#2ecc71', alpha=0.7, edgecolor='black')
axes[2].set_xlabel('PageRank Score', fontsize=12)
axes[2].set_ylabel('Frequency', fontsize=12)
axes[2].set_title('PageRank Distribution', fontsize=14, fontweight='bold')
axes[2].set_yscale('log')
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../models/viz_06_network_centrality.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Saved: viz_06_network_centrality.png")

# Top nodes by centrality
print("\n🔝 Top 5 Nodes by Degree Centrality:")
top_degree = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:5]
for node, score in top_degree:
    print(f"   {node[:10]}... : {score:.6f}")

### 📈 Visualization 3.2: Network Visualization (Interactive)

In [ ]:
# Create subgraph of most connected nodes for visualization
top_nodes = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:50]
top_node_ids = [node for node, _ in top_nodes]
G_sub = G.subgraph(top_node_ids)

# Use spring layout for positioning
pos = nx.spring_layout(G_sub, k=0.5, iterations=50, seed=42)

# Create edge traces
edge_x = []
edge_y = []
for edge in G_sub.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines'
)

# Create node traces
node_x = []
node_y = []
node_text = []
node_size = []

for node in G_sub.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_text.append(f"Address: {node[:10]}...<br>Degree: {G_sub.degree(node)}<br>PageRank: {pagerank.get(node, 0):.6f}")
    node_size.append(20 + 100 * degree_centrality[node])

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    text=node_text,
    marker=dict(
        size=node_size,
        color=[degree_centrality[node] for node in G_sub.nodes()],
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title='Degree Centrality'),
        line_width=2
    )
)

# Create figure
fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='Transaction Network Graph (Top 50 Nodes by Degree Centrality)',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=0, l=0, r=0, t=40),
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    height=700
                ))
fig.show()

print("✅ Interactive network graph displayed")

### 📈 Visualization 3.3: Graph Features Analysis

In [ ]:
# Analyze graph features from processed data
graph_features = ['degree_centrality', 'betweenness_centrality', 'pagerank_score',
                  'clustering_coefficient', 'avg_neighbor_degree', 'triangles',
                  'community_id', 'community_size']

# Filter existing graph features
available_graph_features = [f for f in graph_features if f in df_features.columns]

if available_graph_features:
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    axes = axes.flatten()
    
    for idx, feature in enumerate(available_graph_features):
        if idx < len(axes):
            data = df_features[feature].dropna()
            axes[idx].hist(data, bins=30, color='teal', alpha=0.7, edgecolor='black')
            axes[idx].set_title(f'{feature}', fontsize=12, fontweight='bold')
            axes[idx].set_xlabel('Value')
            axes[idx].set_ylabel('Frequency')
            axes[idx].grid(alpha=0.3)
    
    # Remove empty subplots
    for idx in range(len(available_graph_features), len(axes)):
        fig.delaxes(axes[idx])
    
    plt.tight_layout()
    plt.savefig('../models/viz_07_graph_features.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✅ Saved: viz_07_graph_features.png")
else:
    print("⚠️ No graph features found in processed data")

---
## 🤖 Part 4: Model Training & Performance

Analyze the trained XGBoost model and its performance.

In [ ]:
# Load trained model
import json
from glob import glob

# Find latest model
model_files = sorted(glob(str(MODEL_DIR / "xgb_fraud_*.json")))
if model_files:
    latest_model = model_files[-1]
    model = xgb.XGBClassifier()
    model.load_model(latest_model)
    
    print(f"✅ Loaded model: {Path(latest_model).name}")
    
    # Load parameters
    param_file = latest_model.replace('.json', '').replace('xgb_fraud', 'params') + '.json'
    if Path(param_file).exists():
        with open(param_file, 'r') as f:
            params = json.load(f)
        print(f"\n📊 Model Hyperparameters:")
        for key, value in params['hyperparameters'].items():
            print(f"   {key}: {value}")
else:
    print("⚠️ No trained model found")
    model = None

### 📈 Visualization 4.1: Feature Importance (XGBoost)

In [ ]:
if model is not None:
    # Get feature importance
    importance_types = ['weight', 'gain', 'cover']
    
    fig, axes = plt.subplots(1, 3, figsize=(22, 8))
    
    for idx, imp_type in enumerate(importance_types):
        importance = model.get_booster().get_score(importance_type=imp_type)
        
        if importance:
            # Sort by importance
            importance_sorted = dict(sorted(importance.items(), key=lambda x: x[1], reverse=True)[:15])
            
            axes[idx].barh(list(importance_sorted.keys()), list(importance_sorted.values()), 
                          color='steelblue', alpha=0.8, edgecolor='black')
            axes[idx].set_xlabel(f'Importance ({imp_type})', fontsize=12)
            axes[idx].set_title(f'Top 15 Features by {imp_type.capitalize()}', 
                               fontsize=14, fontweight='bold')
            axes[idx].invert_yaxis()
            axes[idx].grid(alpha=0.3, axis='x')
    
    plt.tight_layout()
    plt.savefig('../models/viz_08_feature_importance_xgboost.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✅ Saved: viz_08_feature_importance_xgboost.png")
else:
    print("⚠️ Model not loaded, skipping feature importance visualization")

---
## 🔍 Part 5: SHAP Explainability Analysis

Analyze SHAP values for model interpretability.

In [ ]:
if model is not None:
    # Prepare data for SHAP
    X_sample = df_features[feature_cols].fillna(0).replace([np.inf, -np.inf], 0)
    sample_size = min(100, len(X_sample))
    X_shap = X_sample.sample(n=sample_size, random_state=42)
    
    # Create SHAP explainer
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_shap)
    
    print(f"✅ Computed SHAP values for {sample_size} samples")
else:
    print("⚠️ Model not loaded, skipping SHAP analysis")
    shap_values = None

### 📈 Visualization 5.1: SHAP Summary Plot

In [ ]:
if shap_values is not None:
    # SHAP summary plot (bar)
    plt.figure(figsize=(12, 8))
    shap.summary_plot(shap_values, X_shap, plot_type="bar", show=False, max_display=20)
    plt.title('SHAP Feature Importance (Global)', fontsize=16, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.savefig('../models/viz_09_shap_summary_bar.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✅ Saved: viz_09_shap_summary_bar.png")

### 📈 Visualization 5.2: SHAP Beeswarm Plot

In [ ]:
if shap_values is not None:
    # SHAP beeswarm plot
    plt.figure(figsize=(12, 10))
    shap.summary_plot(shap_values, X_shap, show=False, max_display=20)
    plt.title('SHAP Beeswarm Plot (Feature Impact)', fontsize=16, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.savefig('../models/viz_10_shap_beeswarm.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✅ Saved: viz_10_shap_beeswarm.png")

### 📈 Visualization 5.3: SHAP Dependence Plots (Top 4 Features)

In [ ]:
if shap_values is not None:
    # Get top 4 features by mean absolute SHAP value
    mean_abs_shap = np.abs(shap_values).mean(axis=0)
    top_features_idx = np.argsort(mean_abs_shap)[-4:][::-1]
    top_features_names = [X_shap.columns[idx] for idx in top_features_idx]
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()
    
    for idx, feature_name in enumerate(top_features_names):
        shap.dependence_plot(
            feature_name, shap_values, X_shap, 
            ax=axes[idx], show=False
        )
        axes[idx].set_title(f'SHAP Dependence: {feature_name}', 
                           fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('../models/viz_11_shap_dependence.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✅ Saved: viz_11_shap_dependence.png")

---
## 📊 Part 6: Model Evaluation Results

Visualize evaluation metrics and results.

In [ ]:
# Load evaluation results
eval_files = sorted(glob(str(MODEL_DIR / "evaluation_results_*.json")))
if eval_files:
    latest_eval = eval_files[-1]
    with open(latest_eval, 'r') as f:
        eval_results = json.load(f)
    
    print(f"✅ Loaded evaluation results: {Path(latest_eval).name}")
    print(f"\n📊 Model Performance Metrics:")
    print(f"   PR-AUC: {eval_results['pr_auc']:.4f}")
    print(f"   ROC-AUC: {eval_results['roc_auc']}")
    print(f"   F1-Score: {eval_results['f1_score']:.4f}")
    print(f"   Precision: {eval_results['precision']:.4f}")
    print(f"   Recall: {eval_results['recall']:.4f}")
    print(f"\n   Confusion Matrix:")
    print(f"   TN: {eval_results['tn']}  FP: {eval_results['fp']}")
    print(f"   FN: {eval_results['fn']}  TP: {eval_results['tp']}")
else:
    print("⚠️ No evaluation results found")
    eval_results = None

### 📈 Visualization 6.1: Performance Metrics Dashboard

In [ ]:
if eval_results:
    # Create metrics dashboard
    metrics = ['PR-AUC', 'F1-Score', 'Precision', 'Recall']
    values = [
        eval_results['pr_auc'],
        eval_results['f1_score'],
        eval_results['precision'],
        eval_results['recall']
    ]
    
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        x=metrics,
        y=values,
        text=[f'{v:.4f}' for v in values],
        textposition='auto',
        marker=dict(
            color=['#3498db', '#2ecc71', '#f39c12', '#e74c3c'],
            line=dict(color='black', width=2)
        )
    ))
    
    fig.update_layout(
        title='Model Performance Metrics',
        xaxis_title='Metric',
        yaxis_title='Score',
        yaxis=dict(range=[0, 1]),
        height=500,
        showlegend=False
    )
    
    fig.show()
    print("✅ Performance metrics dashboard displayed")

### 📈 Visualization 6.2: Display Saved Evaluation Plots

In [ ]:
from IPython.display import Image, display

# Display confusion matrix
cm_files = sorted(glob(str(MODEL_DIR / "confusion_matrix_*.png")))
if cm_files:
    print("📊 Confusion Matrix:")
    display(Image(filename=cm_files[-1]))

# Display PR curve
pr_files = sorted(glob(str(MODEL_DIR / "pr_curve_*.png")))
if pr_files:
    print("\n📊 Precision-Recall Curve:")
    display(Image(filename=pr_files[-1]))

# Display ROC curve
roc_files = sorted(glob(str(MODEL_DIR / "roc_curve_*.png")))
if roc_files:
    print("\n📊 ROC Curve:")
    display(Image(filename=roc_files[-1]))

# Display feature importance
fi_files = sorted(glob(str(MODEL_DIR / "feature_importance_*.png")))
if fi_files:
    print("\n📊 Feature Importance:")
    display(Image(filename=fi_files[-1]))

---
## 📝 Part 7: Summary & Conclusions

In [ ]:
print("="*70)
print("📊 COMPREHENSIVE ANALYSIS SUMMARY")
print("="*70)

print(f"\n📦 Dataset Information:")
print(f"   Total Transactions: {len(df_raw):,}")
print(f"   Unique Addresses: {df_raw['from'].nunique() + df_raw['to'].nunique():,}")
print(f"   Date Range: {df_raw['timestamp'].min()} to {df_raw['timestamp'].max()}")
print(f"   Fraud Rate: {df_raw['is_fraud'].mean() * 100:.2f}%")

print(f"\n🔨 Feature Engineering:")
print(f"   Total Features: {len(feature_cols)}")
print(f"   Feature Categories:")
print(f"      - Temporal: 5")
print(f"      - Value: 4")
print(f"      - Gas: 4")
print(f"      - Account Behavior: 4")
print(f"      - Network Graph: 13")

print(f"\n🕸️ Network Analysis:")
print(f"   Nodes: {G.number_of_nodes():,}")
print(f"   Edges: {G.number_of_edges():,}")
print(f"   Density: {nx.density(G):.6f}")
print(f"   Avg Degree: {sum(dict(G.degree()).values()) / G.number_of_nodes():.2f}")

if eval_results:
    print(f"\n🤖 Model Performance:")
    print(f"   Algorithm: XGBoost")
    print(f"   PR-AUC: {eval_results['pr_auc']:.4f}")
    print(f"   F1-Score: {eval_results['f1_score']:.4f}")
    print(f"   Precision: {eval_results['precision']:.4f}")
    print(f"   Recall: {eval_results['recall']:.4f}")

print(f"\n📁 Generated Visualizations:")
viz_files = [
    "viz_01_value_distribution.png",
    "viz_02_temporal_heatmap.png",
    "viz_03_correlation_matrix.png",
    "viz_04_feature_distributions.png",
    "viz_05_dimensionality_reduction.png",
    "viz_06_network_centrality.png",
    "viz_07_graph_features.png",
    "viz_08_feature_importance_xgboost.png",
    "viz_09_shap_summary_bar.png",
    "viz_10_shap_beeswarm.png",
    "viz_11_shap_dependence.png"
]

for viz_file in viz_files:
    if (MODEL_DIR / viz_file).exists():
        print(f"   ✅ {viz_file}")

print("\n" + "="*70)
print("✅ ANALYSIS COMPLETE!")
print("="*70)

---
## 🎯 Key Findings & Recommendations

### Findings:

1. **Data Distribution**: The dataset shows typical Ethereum transaction patterns with value and gas price following log-normal distributions.

2. **Temporal Patterns**: Transaction activity shows clear temporal patterns with peaks during business hours.

3. **Network Structure**: The transaction network exhibits scale-free properties with a few highly connected hubs.

4. **Feature Importance**: Network graph features (centrality, PageRank) show significant predictive power for fraud detection.

5. **Model Interpretability**: SHAP analysis reveals that account behavior and network position are key fraud indicators.

### Recommendations:

1. **Expand Dataset**: Include more labeled fraud transactions for better model training
2. **Feature Engineering**: Explore additional temporal and network-based features
3. **Model Tuning**: Experiment with different hyperparameters and ensemble methods
4. **Real-time Detection**: Implement streaming analysis for live fraud detection
5. **Community Detection**: Leverage community structure for fraud cluster identification

---

**End of Analysis Notebook**